In [1]:
%matplotlib inline

In [72]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf

from tensorflow.keras.applications import resnet50
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Dropout, Conv2D, BatchNormalization, MaxPool2D, GlobalAvgPool2D, Flatten, Concatenate, Add

# Neural Network Architectures

## Demos

In [3]:
model = resnet50.ResNet50()

In [4]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

                                                                                                  
 conv2_block3_1_relu (Activatio  (None, 56, 56, 64)  0           ['conv2_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block3_2_conv (Conv2D)   (None, 56, 56, 64)   36928       ['conv2_block3_1_relu[0][0]']    
                                                                                                  
 conv2_block3_2_bn (BatchNormal  (None, 56, 56, 64)  256         ['conv2_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_2_relu (Activatio  (None, 56, 56, 64)  0           ['conv2_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv3_block3_1_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block3_2_conv (Conv2D)   (None, 28, 28, 128)  147584      ['conv3_block3_1_relu[0][0]']    
                                                                                                  
 conv3_block3_2_bn (BatchNormal  (None, 28, 28, 128)  512        ['conv3_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block3_2_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv4_block2_1_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block2_1_relu (Activatio  (None, 14, 14, 256)  0          ['conv4_block2_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block2_2_conv (Conv2D)   (None, 14, 14, 256)  590080      ['conv4_block2_1_relu[0][0]']    
                                                                                                  
 conv4_block2_2_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block2_2_conv[0][0]']    
 ization) 

 conv4_block5_1_conv (Conv2D)   (None, 14, 14, 256)  262400      ['conv4_block4_out[0][0]']       
                                                                                                  
 conv4_block5_1_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block5_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block5_1_relu (Activatio  (None, 14, 14, 256)  0          ['conv4_block5_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block5_2_conv (Conv2D)   (None, 14, 14, 256)  590080      ['conv4_block5_1_relu[0][0]']    
                                                                                                  
 conv4_blo

                                                                  'conv5_block1_3_bn[0][0]']      
                                                                                                  
 conv5_block1_out (Activation)  (None, 7, 7, 2048)   0           ['conv5_block1_add[0][0]']       
                                                                                                  
 conv5_block2_1_conv (Conv2D)   (None, 7, 7, 512)    1049088     ['conv5_block1_out[0][0]']       
                                                                                                  
 conv5_block2_1_bn (BatchNormal  (None, 7, 7, 512)   2048        ['conv5_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block2_1_relu (Activatio  (None, 7, 7, 512)   0           ['conv5_block2_1_bn[0][0]']      
 n)       

In [10]:
jpeg_file = tf.io.read_file("./data/cat-1045782_960_720.jpg")
cat_image = tf.image.decode_jpeg(jpeg_file)
cat_image_resized = tf.cast(tf.image.resize(cat_image, (224, 224)), tf.int64)

In [12]:
cat_image_preprocessed = resnet50.preprocess_input(cat_image_resized)

In [15]:
cat_image_preprocessed

<tf.Tensor: shape=(224, 224, 3), dtype=int64, numpy=
array([[[87, 75, 64],
        [78, 66, 55],
        [85, 73, 62],
        ...,
        [90, 78, 67],
        [94, 82, 71],
        [92, 80, 69]],

       [[84, 72, 61],
        [86, 74, 63],
        [84, 72, 61],
        ...,
        [91, 79, 68],
        [95, 83, 72],
        [94, 82, 71]],

       [[97, 85, 74],
        [86, 74, 63],
        [82, 70, 59],
        ...,
        [90, 78, 67],
        [96, 84, 73],
        [98, 86, 75]],

       ...,

       [[89, 77, 66],
        [92, 80, 69],
        [76, 64, 53],
        ...,
        [85, 73, 62],
        [82, 70, 59],
        [98, 86, 75]],

       [[90, 78, 67],
        [84, 72, 61],
        [70, 58, 47],
        ...,
        [82, 70, 59],
        [81, 69, 58],
        [90, 78, 67]],

       [[90, 78, 67],
        [78, 66, 55],
        [61, 49, 38],
        ...,
        [69, 57, 46],
        [80, 68, 57],
        [71, 59, 48]]], dtype=int64)>

In [17]:
cat_image_batch = tf.expand_dims(cat_image_preprocessed, 0)

In [18]:
cat_image_batch

<tf.Tensor: shape=(1, 224, 224, 3), dtype=int64, numpy=
array([[[[87, 75, 64],
         [78, 66, 55],
         [85, 73, 62],
         ...,
         [90, 78, 67],
         [94, 82, 71],
         [92, 80, 69]],

        [[84, 72, 61],
         [86, 74, 63],
         [84, 72, 61],
         ...,
         [91, 79, 68],
         [95, 83, 72],
         [94, 82, 71]],

        [[97, 85, 74],
         [86, 74, 63],
         [82, 70, 59],
         ...,
         [90, 78, 67],
         [96, 84, 73],
         [98, 86, 75]],

        ...,

        [[89, 77, 66],
         [92, 80, 69],
         [76, 64, 53],
         ...,
         [85, 73, 62],
         [82, 70, 59],
         [98, 86, 75]],

        [[90, 78, 67],
         [84, 72, 61],
         [70, 58, 47],
         ...,
         [82, 70, 59],
         [81, 69, 58],
         [90, 78, 67]],

        [[90, 78, 67],
         [78, 66, 55],
         [61, 49, 38],
         ...,
         [69, 57, 46],
         [80, 68, 57],
         [71, 59, 48]]]], dtype

In [19]:
model.predict(cat_image_batch)

1/1 [==============================] - 1s 816ms/step


array([[1.64396695e-07, 3.17531203e-07, 3.16376401e-08, 3.15624931e-07,
        2.47309117e-07, 1.29136779e-07, 1.54707195e-07, 1.69434554e-06,
        4.32989100e-06, 2.01533367e-05, 9.26119299e-07, 8.07476795e-07,
        1.83515624e-06, 2.43304584e-07, 1.32876232e-06, 9.57742600e-07,
        1.94986640e-07, 5.99462646e-06, 1.16643503e-06, 1.35187648e-07,
        4.73059657e-07, 4.14307306e-06, 1.14065133e-05, 5.73298735e-07,
        1.43489087e-05, 8.49941912e-08, 1.78092023e-07, 1.81575922e-06,
        7.32389793e-08, 3.11637223e-08, 2.25315989e-06, 2.38108919e-06,
        9.06876267e-07, 1.37246303e-08, 1.45034491e-07, 1.22808814e-07,
        3.65068246e-07, 3.06114373e-07, 1.26275154e-06, 2.05113224e-06,
        1.49069635e-07, 2.44618536e-06, 2.48499168e-06, 1.16367846e-06,
        4.55564845e-07, 2.23576308e-07, 1.80555071e-06, 7.40908490e-08,
        6.02042235e-07, 1.56188707e-06, 8.09839307e-08, 1.28037357e-06,
        5.34299318e-07, 8.46026637e-07, 9.03429452e-07, 1.467955

In [22]:
resnet50.decode_predictions(model.predict(cat_image_batch))

1/1 [==============================] - 0s 108ms/step


[[('n02127052', 'lynx', 0.40546808),
  ('n02123045', 'tabby', 0.27712137),
  ('n02123159', 'tiger_cat', 0.24938557),
  ('n02124075', 'Egyptian_cat', 0.023296833),
  ('n02123394', 'Persian_cat', 0.020655464)]]

In [23]:
resnet50.decode_predictions(model.predict(cat_image_batch), top = 1)

1/1 [==============================] - 0s 78ms/step


[[('n02127052', 'lynx', 0.40546808)]]

In [26]:
resnet50.decode_predictions(model.predict(cat_image_batch), top = 5)

1/1 [==============================] - 0s 96ms/step


[[('n02127052', 'lynx', 0.40546808),
  ('n02123045', 'tabby', 0.27712137),
  ('n02123159', 'tiger_cat', 0.24938557),
  ('n02124075', 'Egyptian_cat', 0.023296833),
  ('n02123394', 'Persian_cat', 0.020655464)]]

In [27]:
model.layers

In [28]:
model.layers[: -1]

In [33]:
model2 = Model(inputs = model.input, outputs = model.output)

In [34]:
model2.output_shape

(None, 1000)

In [35]:
model2.input_shape

(None, 224, 224, 3)

In [38]:
conv_head = Model(inputs = model.input, outputs = model.layers[-2].output)

In [39]:
conv_head.input_shape, conv_head.output_shape

((None, 224, 224, 3), (None, 2048))

In [56]:
transfer_model = Sequential([
    conv_head,
    Dense(256, activation = "relu"),
    Dropout(0.1),
    Dense(8, activation = "softmax")
])

In [57]:
transfer_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model_3 (Functional)        (None, 2048)              23587712  
                                                                 
 dense_13 (Dense)            (None, 256)               524544    
                                                                 
 dropout_4 (Dropout)         (None, 256)               0         
                                                                 
 dense_14 (Dense)            (None, 8)                 2056      
                                                                 
Total params: 24,114,312
Trainable params: 24,061,192
Non-trainable params: 53,120
_________________________________________________________________


In [52]:
dense_layer_intermediate = Dense(256, activation = "relu")(conv_head.output)
dropuot_intermediate = Dropout(0.1)(dense_layer_intermediate)
dense_layer_output = Dense(8, activation = "softmax")(dropuot_intermediate)

In [53]:
dense_layer_output

<KerasTensor: shape=(None, 8) dtype=float32 (created by layer 'dense_8')>

In [67]:
for i, layer in enumerate(conv_head.layers):
    print(i, layer)

0 <keras.engine.input_layer.InputLayer object at 0x000002BB79853400>
1 <keras.layers.reshaping.zero_padding2d.ZeroPadding2D object at 0x000002BB044E9A00>
2 <keras.layers.convolutional.conv2d.Conv2D object at 0x000002BB044E9F70>
3 <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x000002BB0456ED30>
4 <keras.layers.core.activation.Activation object at 0x000002BB0456E2E0>
5 <keras.layers.reshaping.zero_padding2d.ZeroPadding2D object at 0x000002BB045B9D30>
6 <keras.layers.pooling.max_pooling2d.MaxPooling2D object at 0x000002BB045FE640>
7 <keras.layers.convolutional.conv2d.Conv2D object at 0x000002BB046075E0>
8 <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x000002BB04617C70>
9 <keras.layers.core.activation.Activation object at 0x000002BB04617490>
10 <keras.layers.convolutional.conv2d.Conv2D object at 0x000002BB0461D910>
11 <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x000002BB04623640>
12 <keras.lay

In [68]:
for layer in model.layers[:161]:
    layer.trainable = False

In [69]:
transfer_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model_3 (Functional)        (None, 2048)              23587712  
                                                                 
 dense_13 (Dense)            (None, 256)               524544    
                                                                 
 dropout_4 (Dropout)         (None, 256)               0         
                                                                 
 dense_14 (Dense)            (None, 8)                 2056      
                                                                 
Total params: 24,114,312
Trainable params: 6,046,984
Non-trainable params: 18,067,328
_________________________________________________________________


In [71]:
block1 = Sequential([
    Conv2D(32, kernel_size = (3, 3), activation = "relu"),
    Conv2D(32, kernel_size = (3, 3), activation = "relu"),
    MaxPool2D()
])

block2 = Sequential([
    Conv2D(64, kernel_size = (3, 3), activation = "relu"),
    Conv2D(64, kernel_size = (3, 3), activation = "relu"),
    MaxPool2D()
])

block3 = Sequential([
    Conv2D(64, kernel_size = (5, 5), activation = "relu"),
    Conv2D(64, kernel_size = (5, 5), activation = "relu"),
    MaxPool2D()
])

In [77]:
Sequential([
    Input(shape = (224, 224, 3)),
    block1,
    block2,
    block3
]).summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_4 (Sequential)   (None, 110, 110, 32)      10144     
                                                                 
 sequential_5 (Sequential)   (None, 53, 53, 64)        55424     
                                                                 
 sequential_6 (Sequential)   (None, 22, 22, 64)        204928    
                                                                 
Total params: 270,496
Trainable params: 270,496
Non-trainable params: 0
_________________________________________________________________


In [100]:
tf.keras.backend.clear_session()

In [101]:
input_l = Input(shape = (42, ))
dense1 = Dense(32, activation = "relu")(input_l)
dense2_1 = Dense(64, activation = "relu")(dense1)
dense2_2 = Dense(42, activation = "relu")(dense1)
concat = Concatenate()([dense2_1, dense2_2])

In [102]:
concat

<KerasTensor: shape=(None, 106) dtype=float32 (created by layer 'concatenate')>

In [103]:
input_l = Input(shape = (42, ))
dense1 = Dense(106, activation = "relu")(input_l)
dense2_1 = Dense(64, activation = "relu")(dense1)
dense2_2 = Dense(42, activation = "relu")(dense1)
concat = Concatenate()([dense2_1, dense2_2])
add = Add()([dense1, concat])

In [104]:
my_new_model = Model(inputs = input_l, outputs = add)

In [105]:
my_new_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 42)]         0           []                               
                                                                                                  
 dense_3 (Dense)                (None, 106)          4558        ['input_2[0][0]']                
                                                                                                  
 dense_4 (Dense)                (None, 64)           6848        ['dense_3[0][0]']                
                                                                                                  
 dense_5 (Dense)                (None, 42)           4494        ['dense_3[0][0]']                
                                                                                              

In [107]:
class MyCallback(tf.keras.callbacks.Callback):
    def __init__(self,...):
        super(...)

SyntaxError: invalid syntax (4203566945.py, line 2)